In [ ]:
%load_ext autoreload
%autoreload 2

Репо: https://huggingface.co/SleMary/my-new-repo/tree/main    

In [ ]:
!pip install -q datasets hydra-core evaluate trl accelerate bitsandbytes peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 8.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Hydra

In [ ]:
import os

import hydra
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from hydra.core.global_hydra import GlobalHydra
from hydra.core.config_store import ConfigStore
from omegaconf import OmegaConf

In [ ]:
with initialize_config_dir(version_base=None,
                           config_dir="/content/drive/MyDrive/hydra_configs/fine_tune",
                           job_name="fine_tune"):
    config = compose(config_name='config.yaml')

# Mistral

## Load model

In [ ]:
from huggingface_hub import login
login(token=config.hf.token)

In [ ]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 19.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch


tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", token=config.hf.token)
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", token=config.hf.token)
model.gradient_checkpointing_enable()

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

## Push to hub

In [ ]:
model.push_to_hub(config.hf.target_model_id, token=config.hf.token)
tokenizer.push_to_hub(config.hf.target_model_id, token=config.hf.token)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

model-00002-of-00006.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/SleMary/mistral-7b-v0.3-instruct-norobots/commit/8646c5914820fdc2787961151d1f178c279785ba', commit_message='Upload tokenizer', commit_description='', oid='8646c5914820fdc2787961151d1f178c279785ba', pr_url=None, repo_url=RepoUrl('https://huggingface.co/SleMary/mistral-7b-v0.3-instruct-norobots', endpoint='https://huggingface.co', repo_type='model', repo_id='SleMary/mistral-7b-v0.3-instruct-norobots'), pr_revision=None, pr_num=None)

## Install libraries

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp.git
!pip install -r llama.cpp/requirements.txt
!pip install huggingface_hub

Cloning into 'llama.cpp'...
remote: Enumerating objects: 47091, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 47091 (delta 52), reused 29 (delta 26), pack-reused 47015 (from 2)
Receiving objects: 100% (47091/47091), 99.21 MiB | 37.81 MiB/s, done.
Resolving deltas: 100% (33816/33816), done.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.8/186.8 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of transformers to determ

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 16.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00
ERROR: Operation cancelled by user
^C


## Download model

In [ ]:
from huggingface_hub import snapshot_download
snapshot_download(repo_id=config.hf.target_model_id,
                  local_dir="mistral-norobots",
                  local_dir_use_symlinks=False,
                  revision="main")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:834: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

...

model-00002-of-00006.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.67M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

'/content/mistral-norobots'

## Quantize to 8bit

In [ ]:
!pip install llama-cpp-python

In [ ]:
!python llama.cpp/convert_hf_to_gguf.py /content/mistral-norobots --outtype q8_0

INFO:hf-to-gguf:Loading model: mistral-norobots
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00006.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.float32 --> Q8_0, shape = {4096, 32768}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.float32 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.float32 --> Q8_0, shape = {14336, 4096}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float32 --> Q8_0, shape = {4096, 14336}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float32 --> Q8_0, shape = {4096, 14336}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.float32 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.float32 --> Q8_0, shape = {4096, 1024}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.float32 --> Q8_0, 

## Save

In [ ]:
from llama_cpp import Llama


llm = Llama(
    model_path="/content/mistral-norobots/Mistral-7B-Instruct-v0.3-Q8_0.gguf"
)
output = llm(
      "Q: Name the planets in the solar system? A: ", # Prompt
      max_tokens=32, # Generate up to 32 tokens, set to None to generate up to the end of the context window
      stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
      echo=True # Echo the prompt back in the output
) # Generate a completion, can also call create_completion
print(output)

ModuleNotFoundError: No module named 'llama_cpp'

In [ ]:
from huggingface_hub import create_repo
create_repo("my-new-repo")

In [ ]:
from huggingface_hub import HfApi
api = HfApi()


api.upload_file(
    path_or_fileobj="/content/mistral-norobots/Mistral-Norobots-7.2B-Q8_0.gguf",
    path_in_repo="Mistral-7B-Instruct-v0.3-Q8_0.gguf",
    repo_id='SleMary/my-new-repo',
    token=config.hf.token
)

Mistral-Norobots-7.2B-Q8_0.gguf:   0%|          | 0.00/7.70G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/SleMary/my-new-repo/commit/4688d09447ef0850251025cc03cd97a54b19a6fa', commit_message='Upload Mistral-7B-Instruct-v0.3-Q8_0.gguf with huggingface_hub', commit_description='', oid='4688d09447ef0850251025cc03cd97a54b19a6fa', pr_url=None, repo_url=RepoUrl('https://huggingface.co/SleMary/my-new-repo', endpoint='https://huggingface.co', repo_type='model', repo_id='SleMary/my-new-repo'), pr_revision=None, pr_num=None)

Можно так же считывать модели при помощи функционала из transformers и и квантизовывать их при помощи quantize.  
Но для этого нужно подходящее окружение.  
https://huggingface.co/docs/transformers/main/quantization/awq

In [ ]:
!pip install -q transformers accelerate
# !pip install -q -U https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.6/autoawq-0.1.6+cu118-cp310-cp310-linux_x86_64.whl

ERROR: autoawq-0.1.6+cu118-cp310-cp310-linux_x86_64.whl is not a supported wheel on this platform.


In [ ]:
!pip install autoawq

  Using cached autoawq-0.2.8.tar.gz (71 kB)
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_path = "facebook/opt-125m"
quant_path = "opt-125m-awq"
quant_config = {"zero_point": True, "q_group_size": 128, "w_bit": 4, "version":"GEMM"}

# Load model
model = AutoAWQForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Quantize
model.quantize(tokenizer, quant_config=quant_config)

ModuleNotFoundError: No module named 'awq'